# Librerias necesarias para manipular, visualizar y operar datos

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# creacion del tablero de juego en el que se va a simular

In [4]:

# Usamos un diccionario o hash table para crear los cuadros del tablero que
# avanzan desde 1 a 36.
tablero = {k:k for k in range(1,37)}


# Definiendo los cuadros que representan una escalera alterando la dinamica
# si un jugador cae en la casilla de las llaves del diccionario, su valor final
# ascendera al respectivo en la tabla.
tablero[3] = 16
tablero[5] = 7
tablero[15] = 25
tablero[18] = 20
tablero[21] = 32


# Definiendo los cuadros que representan una serpiente alterando la dinamica
# si un jugador cae en la casilla de las llaves del diccionario, su valor final 
# descendera al respectivo en la tabla.
tablero[12] = 2
tablero[14] = 11
tablero[17] = 4
tablero[31] = 19
tablero[35] = 22



# Ordenando el tablero ascendentemente.
tablero = {k:v for k,v in sorted(tablero.items())}

# creacion de una partida de dos jugadores
con todas las reglas sugeridas, utilizando un sistema de numeros aleatorios equiprobable anologo a los dados y
con posible configuracion para todos los modos justos de partida que el simulador solicite

In [5]:
def juego_1_vs_1(adelantar_posicion_de_jugador_2=False,desactivar_primera_serpiente=False,posicion_de_adelanto_jugador_2=0):

    jugador_1_posicion = 0
    jugador_2_posicion = posicion_de_adelanto_jugador_2
    n_de_serpientes_pasadas = 0
    n_de_lanzamiento_de_dados_para_ganar = 0

    while True:
        n_de_lanzamiento_de_dados_para_ganar +=1
        
        
        if adelantar_posicion_de_jugador_2 and jugador_1_posicion == jugador_2_posicion and jugador_1_posicion==36:
            return "empate",n_de_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar

        elif jugador_1_posicion == 36:
            return "gano jugador 1",n_de_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar
        
        elif jugador_2_posicion == 36:
            return "gano jugador 2",n_de_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar
        
        else:
            dado_1 = np.random.choice(range(1,7),p=[1/6]*6)
            dado_2 = np.random.choice(range(1,7),p=[1/6]*6)
            
            avance_lineal_jugador_1 = jugador_1_posicion+dado_1
            avance_lineal_jugador_2 = jugador_2_posicion+dado_2

            if desactivar_primera_serpiente and avance_lineal_jugador_2 == 12:
                tablero[12] = 12
            elif desactivar_primera_serpiente and avance_lineal_jugador_1 == 12:
                tablero[12] = 2
            
            serpientes = [12,14,17,31,35]
            
            if avance_lineal_jugador_1 in serpientes or avance_lineal_jugador_2 in serpientes:
                n_de_serpientes_pasadas+=1 

            if avance_lineal_jugador_1<36:
                jugador_1_turno = tablero[avance_lineal_jugador_1]
            else:
                jugador_1_turno = 36
            if avance_lineal_jugador_2<36: 
                jugador_2_turno = tablero[avance_lineal_jugador_2]
            else: 
                jugador_2_turno = 36 
            
            jugador_1_posicion = jugador_1_turno
            jugador_2_posicion = jugador_2_turno
            
            continue

# Funcion de simulacion de n partidas
<br> en este caso simulamos diez mil partidas que retornan el historial de partidas donde se almacena informacion como
 cuantas victorias logro el jugador 1 y 2; asi como empates y derrotas.</br>
Tambien mide cuantas veces ambos jugadores cayeron en serpientes por total de partida, 
 asi como el numero de lanzamientos de dados necesarios para que uno de los jugadores gane
 una partida

In [6]:
def simulacion_de_diez_mil_partidas(desactivar_primera_serpiente=False,adelantar_posicion_de_jugador_2=False,posicion_de_adelanto_jugador_2=0):
  
    historial = pd.DataFrame(index=["victorias_1","victorias_2",
                                    "empates","derrotas_1","derrotas_2","n_serpientes","n_de_lanzamientos_para_ganar"])

    victorias_jugador_1 = 0
    empates = 0
    victorias_jugador_2 = 0

    for a in range(10000):
        partida,n_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar = juego_1_vs_1(desactivar_primera_serpiente=desactivar_primera_serpiente,
                                                                                        adelantar_posicion_de_jugador_2=adelantar_posicion_de_jugador_2,
                                                                                        posicion_de_adelanto_jugador_2=posicion_de_adelanto_jugador_2)

        if partida == "gano jugador 1":
            historial[a] = [1,0,0,0,1,n_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar]
            victorias_jugador_1+=1
        
        elif partida == "gano jugador 2":
            historial[a] = [0,1,0,1,0,n_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar]
            victorias_jugador_2+=1
        
        else:
            historial[a] = [0,0,1,0,0,n_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar]
            empates += 1


    return historial.T

# Simulacion de una partida en la que el jugador solo cae en una escalera
donde puede caer el 50 % de las veces en el pie de la escalera lo que lo ascenderia o en el fin de la misma lo cual lo dejaria en el mismo lugar.

In [34]:
def juego_cayendo_solo_escaleras():

    posicion = np.random.choice([3,5],p=[0.5,0.5])
    n_de_lanzamiento_de_dados_para_ganar = 0

    while True:
        n_de_lanzamiento_de_dados_para_ganar +=1
        inicio_de_escaleras = [3,5,15,18,21]
        fin_de_escaleras = [tablero[pp] for pp in inicio_de_escaleras]
        caer_en_escalera = inicio_de_escaleras+fin_de_escaleras
        caer_en_escalera = sorted(caer_en_escalera)
        
        df = pd.Series(caer_en_escalera)

        if posicion+6>= 36:
            return n_de_lanzamiento_de_dados_para_ganar

        opciones_de_avance = df[(df>posicion)&(df<=posicion+9)]
        siguiente_escalera = np.random.choice(opciones_de_avance.index)
        posicion = df[df.index==siguiente_escalera].values[0]


# Probando mil partidas de la simulacion de solo escaleras

In [35]:
veces = [juego_cayendo_solo_escaleras() for a in range(10000)]
veces = np.array(veces)

veces.mean()

6.5872

# Probabilidad de que el jugado numero 1 gane en base a una simulacion de 10000 partidas

In [10]:
resultado = pd.DataFrame(index=["probabilidad_de_victoria_jugador_1","probabilidad_de_victoria_jugador_2","probabilidad_de_empate",
                                "promedio_de_caidas_en_Serpiente","desviacion_de_caidas_en_serpiente",
                                "promedio_de_lanzamientos_para_ganar","desviacion_de_lanzamientos_para_ganar"])

modos = ["normal","jugador_2_adelantado","jugador_2_adelantado","jugador_2_sin_primera_serpiente"]

for i,modo in enumerate(modos):

    if modo=="normal":
        historial = simulacion_de_diez_mil_partidas()
    elif modo=="jugador_2_adelantado":
        modo = modo+f"_{i+4}"
        historial = simulacion_de_diez_mil_partidas(posicion_de_adelanto_jugador_2=i+4)
    else:
        historial = simulacion_de_diez_mil_partidas(desactivar_primera_serpiente=True)

    probabilidad_de_victoria_jugador_1 = historial.victorias_1.sum()/historial.shape[0]
    probabilidad_de_victoria_jugador_2 = historial.victorias_2.sum()/historial.shape[0]
    probabilidad_de_empate = historial.empates.sum()/historial.shape[0]
    promedio_de_caidas_en_serpiente = historial.n_serpientes.mean()
    desviacion_de_caidas_en_serpiente = historial.n_serpientes.std()
    promedio_de_lanzamientos_para_ganar = historial.n_de_lanzamientos_para_ganar.mean()
    desviacion_de_lanzamientos_para_ganar = historial.n_de_lanzamientos_para_ganar.std()
        
    valores = [probabilidad_de_victoria_jugador_1,probabilidad_de_victoria_jugador_2,probabilidad_de_empate,
            promedio_de_caidas_en_serpiente,desviacion_de_caidas_en_serpiente,
            promedio_de_lanzamientos_para_ganar,desviacion_de_lanzamientos_para_ganar]

    resultado[modo] = valores

    print(valores)

resultado.T

/tmp/ipykernel_158/1937841432.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  historial[a] = [0,1,0,1,0,n_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar]
/tmp/ipykernel_158/1937841432.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  historial[a] = [1,0,0,0,1,n_serpientes_pasadas,n_de_lanzamiento_de_dados_para_ganar]


[0.5261, 0.4739, 0.0, 2.806, 2.246523689362898, 10.8031, 4.109552193600075]
[0.5144, 0.4856, 0.0, 3.0045, 2.2580942515681, 10.6902, 3.9495548937045855]
[0.5161, 0.4839, 0.0, 3.1207, 2.280537567144974, 10.7558, 4.020395840992753]
[0.4907, 0.5093, 0.0, 2.7648, 2.1635501045516583, 10.499, 3.8023472729984102]


,probabilidad_de_victoria_jugador_1,probabilidad_de_victoria_jugador_2,probabilidad_de_empate,promedio_de_caidas_en_Serpiente,desviacion_de_caidas_en_serpiente,promedio_de_lanzamientos_para_ganar,desviacion_de_lanzamientos_para_ganar
normal,0.5261,0.4739,0.0,2.8060,2.246524,10.8031,4.109552
jugador_2_adelantado_5,0.5144,0.4856,0.0,3.0045,2.258094,10.6902,3.949555
jugador_2_adelantado_6,0.5161,0.4839,0.0,3.1207,2.280538,10.7558,4.020396
jugador_2_sin_primera_serpiente,0.4907,0.5093,0.0,2.7648,2.163550,10.4990,3.802347


In [11]:
resultado = resultado.T

1.- En una partida de dos personas, cual es la probabiliad que el jugador que empieza gane el juego

In [13]:
resultado.iloc[0,0]

0.5261

In [14]:
resultado

,probabilidad_de_victoria_jugador_1,probabilidad_de_victoria_jugador_2,probabilidad_de_empate,promedio_de_caidas_en_Serpiente,desviacion_de_caidas_en_serpiente,promedio_de_lanzamientos_para_ganar,desviacion_de_lanzamientos_para_ganar
normal,0.5261,0.4739,0.0,2.8060,2.246524,10.8031,4.109552
jugador_2_adelantado_5,0.5144,0.4856,0.0,3.0045,2.258094,10.6902,3.949555
jugador_2_adelantado_6,0.5161,0.4839,0.0,3.1207,2.280538,10.7558,4.020396
jugador_2_sin_primera_serpiente,0.4907,0.5093,0.0,2.7648,2.163550,10.4990,3.802347


2.- En promedio, cuantas veces se cae en una serpiente en cada juego 

In [15]:
resultado.iloc[0,3]

2.806

3.- Si cada vez un jugador llega a una escalera y solo hay un 50% de probabilidad de que la pueda tomar, cual es el numero promedio de intentos necesarios para completar el juego

In [16]:
veces.mean()

6.4922

4.- Empezando con el juego base, decides que quieres que el juego tenga apromixamadamente
las mismas probabilidades. Haces esto cambiando el cuadrado en el que el jugador dos empieza.
En cual cuadrado debe empezar el jugador dos para que se obtenga una probabilidad justa para ambos.

In [29]:
resultado.iloc[2,1]

0.4839

In [27]:
resultado.iloc[:,1].index[2]

'jugador_2_adelantado_6'

5.- En un intento diferente para cambiar las chances de el juego, en vez de empezar avanzado el jugador 2 en un cuadrado diferente, decides darle al jugador 2 inmunidad a la primera serpiente en la que caiga. Cual es la la probabilidad aproximada que el jugador 1 gane ahora

In [33]:
resultado.iloc[-1,1]

0.5093